In [ ]:
import torch
import preprocess
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm

In [ ]:
# 设置任务和模型
task_name = "STS-B" # ["CoLA", "SST-2", "MRPC", "STS-B", "QQP", "MNLI-m", "MNLI-mm", "QNLI", "RTE", "WNLI"]
model_name = "models/bert-base-uncased" # [bert-base-cased, bert-base-uncased, roberta-base]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# 设置超参数
# 训练参数
SEED = 0
BATCH_SIZE = 8
EPOCH = 10
LEARNING_RATE = 1e-5
# 对抗训练参数
Adv_step = 5
Adv_epsilon = 5e-2
Adv_max_norm = 0  # 0表示不限制扰动大小
Adv_lambda = 1
Adv_mu = 1

In [ ]:
model, dataloader, metric= preprocess.preprocess(task_name, model_name, BATCH_SIZE, SEED)
train_dataloader,eval_dataloader,test_dataloader = dataloader

In [ ]:
# 设置优化器
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
# ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"]
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=EPOCH * len(train_dataloader))

In [ ]:
import torch.nn.functional as F
from torch.nn import MSELoss

def ls(P, Q, task_type):
    if(task_type == "classification"):
        return F.kl_div(P.softmax(dim=-1).log(), Q.softmax(dim=-1), reduction='sum') + F.kl_div(Q.softmax(dim=-1).log(), P.softmax(dim=-1), reduction='sum')
    elif(task_type == "regression"):
        return MSELoss(P, Q, reduction="sum")


In [ ]:
# Training
print("*"*20, "Training", "*"*20)  # 训练参数
print("TASK:", task_name)
print("MODEL:", model_name)
print("DEVICE:", device)
print("-"*50)
print("EPOCH_NUM:", EPOCH)
print("BATCH_SIZE:", BATCH_SIZE)
print("LEARNING_RATE:", LEARNING_RATE)
print("="*14, "Adversarial Training", "="*14)  # 对抗训练参数
print("Adversarial_Training_type:","SMART")
print("Adversarial_step:", Adv_step)
print("Adversarial_epsilon:", Adv_epsilon)
print("Adversarial_max_norm:", Adv_max_norm)
print("*"*50)
model.to(device)
old_parameters = model.parameters()
progress_bar = tqdm(range(EPOCH * len(train_dataloader)))
for i in range(EPOCH):
    print("-"*20, "EPOCH:", i, "-"*20)
    print("Training...")
    model.train()
    for batch in train_dataloader:
        for t in batch:
            batch[t] = batch[t].to(device)
        
        # [begin] Adversarial Training
        # 1.get L-loss
        L_outputs=model(**batch)
        L_loss = L_outputs.loss

        # 2.get R-loss
        ## 2.1 init delta
        if "bert-base" in model_name:
            word_embedding_init = model.bert.embeddings.word_embeddings(batch["input_ids"])
        elif "roberta-base" in model_name:
            word_embedding_init = model.roberta.embeddings.word_embeddings(batch["input_ids"])
        delta = torch.zeros_like(word_embedding_init)
        adv_inputs = {"attention_mask" : batch["attention_mask"], "labels" : batch["labels"], "token_type_ids" : batch["token_type_ids"]}
        ## 2.2 updata delta like PGD
        for step in range(Adv_step):
            delta.requires_grad = True
            word_embedding = word_embedding_init.clone().detach()
            adv_inputs["inputs_embeds"] = delta + word_embedding # 「inputs_embeds」和「input_ids」参数二选一，使用添加扰动的「inputs_embeds」
            outputs = model(**adv_inputs)
            loss = outputs.loss
            loss.backward() 

            delta_grad = delta.grad.clone().detach() # delta的梯度
            delta_grad_norm = torch.norm(delta_grad.view(delta_grad.size(0), -1), dim=1).view(-1, 1, 1) # .view：摊平  .norm：求L2范数  .view：重新设置shape
            delta_grad_norm = torch.clamp(delta_grad_norm, min=1e-8)  # 设置最小值，避免下一步除数为0
            delta = (delta + Adv_epsilon * delta_grad / delta_grad_norm).detach()  # 新的扰动 δ = δ + ε * g/||g||

            if Adv_max_norm > 0:  # 限制扰动大小  Adv_max_norm = 0 则不限制
                delta_norm = torch.norm(delta.view(delta.size(0), -1).float(), p=2, dim=1).detach()
                exceed_mask = (delta_norm > Adv_max_norm).to(delta_norm)
                if(sum(exceed_mask) != 0):  # 存在超出限制的扰动大小
                    reweights = (Adv_max_norm / delta_norm * exceed_mask + (1-exceed_mask)).view(-1, 1, 1) # 缩减比例
                    delta = (delta * reweights).detach()  # 按比例缩减到norm-ball内
        ## 2.3 put the final delta into the model
        adv_inputs["inputs_embeds"] = delta + word_embedding_init.clone().detach()
        R_outputs = model(**adv_inputs)
        R_loss = ls(R_outputs, L_outputs)

        # 3.get D-loss
        for p in model.parameters():
            p.data = old_parameters
        D_outputs=model(**batch)
        D_loss = ls(L_outputs, D_outputs)
        old_parameters = model.parameters()

        # 4.optimize model parameters
        loss = L_loss + Adv_lambda * R_loss + Adv_mu * D_loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        # [end] Adversarial Training

    print("Eval...")
    model.eval()
    for batch in eval_dataloader:
        for t in batch:
            batch[t] = batch[t].to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1) if task_name != "STS-B" else outputs.logits.squeeze()
        metric.add_batch(predictions=predictions, references=batch["labels"])
    print("Metric:", metric.compute())
    print("-"*50)
